In [1]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [2]:
#change path
path="..\\UB_keystroke_dataset\\S0"

In [3]:
def ReadListOfTextFilesFromDirectoryWalk(path): 
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r, file))
    return files



In [4]:
files=ReadListOfTextFilesFromDirectoryWalk(path)
len(files)
print(files[0:2])

['..\\UB_keystroke_dataset\\S0\\baseline\\001000.txt', '..\\UB_keystroke_dataset\\S0\\baseline\\001001.txt']


In [41]:
def CreateListOfTuplesFromFile(files):
#Reading file and create tuple 
    items = []
    iterator = (files,) if not isinstance(files, (tuple, list)) else files
    for file in iterator:
        filename=file.split('\\')[-1].split('.')[0]
        f=open(file, "r")
        for line in f:
            words= line.rstrip('\n').split(sep=" ")
            items.append((words[0],words[1],words[2],filename))
        f.close()
    return items


In [6]:
items=CreateListOfTuplesFromFile(files[0:2])
items

[('LMenu', 'KeyDown', '63578429757751', '001000'),
 ('Tab', 'KeyDown', '63578429757861', '001000'),
 ('LMenu', 'KeyUp', '63578429757970', '001000'),
 ('Tab', 'KeyUp', '63578429757985', '001000'),
 ('LMenu', 'KeyDown', '63578429761511', '001000'),
 ('Tab', 'KeyDown', '63578429761589', '001000'),
 ('LMenu', 'KeyUp', '63578429761698', '001000'),
 ('Tab', 'KeyUp', '63578429761729', '001000'),
 ('LMenu', 'KeyDown', '63578429762229', '001000'),
 ('Tab', 'KeyDown', '63578429762291', '001000'),
 ('LMenu', 'KeyUp', '63578429762416', '001000'),
 ('Tab', 'KeyUp', '63578429762447', '001000'),
 ('LMenu', 'KeyDown', '63578429762837', '001000'),
 ('Tab', 'KeyDown', '63578429762931', '001000'),
 ('Tab', 'KeyUp', '63578429763071', '001000'),
 ('Tab', 'KeyDown', '63578429763789', '001000'),
 ('Tab', 'KeyUp', '63578429763882', '001000'),
 ('LMenu', 'KeyUp', '63578429764085', '001000'),
 ('D0', 'KeyDown', '63578429768578', '001000'),
 ('D0', 'KeyUp', '63578429768656', '001000'),
 ('D0', 'KeyDown', '635784

In [7]:
#Creating dataframes
def CreateDataFrame(items,columns):
    df = pd.DataFrame.from_records(items, columns=['Key', 'EventType','Time','User'])
    return df

In [8]:
df=CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
print (df.shape)
#
df.head(10)

(13900, 4)


,Key,EventType,Time,User
0,LMenu,KeyDown,63578429757751,001000
1,Tab,KeyDown,63578429757861,001000
2,LMenu,KeyUp,63578429757970,001000
3,Tab,KeyUp,63578429757985,001000
4,LMenu,KeyDown,63578429761511,001000
5,Tab,KeyDown,63578429761589,001000
6,LMenu,KeyUp,63578429761698,001000
7,Tab,KeyUp,63578429761729,001000
8,LMenu,KeyDown,63578429762229,001000
9,Tab,KeyDown,63578429762291,001000


In [9]:
def ParseAlphabetsKeys(df):
    dfAlphabets=df[df["Key"].str.match('^.*[A-Z]$')]
    return dfAlphabets

In [10]:
temp=ParseAlphabetsKeys(df)
temp.groupby("Key").size()/2

Key
A    375.0
B     76.0
C    137.0
D    217.0
E    585.0
F    119.0
G     99.0
H    203.0
I    361.0
J      7.0
K     37.0
L    206.0
M    117.0
N    286.0
O    363.0
P     92.0
Q      4.0
R    262.0
S    322.0
T    453.0
U    149.0
V     57.0
W     92.0
X      6.0
Y    112.0
Z      2.0
dtype: float64

In [11]:
temp

,Key,EventType,Time,User
47,I,KeyDown,63578429792602,001000
48,I,KeyUp,63578429792696,001000
52,A,KeyDown,63578429792961,001000
53,A,KeyUp,63578429793054,001000
54,M,KeyDown,63578429793257,001000
55,M,KeyUp,63578429793382,001000
58,H,KeyDown,63578429793616,001000
59,H,KeyUp,63578429793741,001000
60,O,KeyDown,63578429793756,001000
61,O,KeyUp,63578429793881,001000


In [12]:
def GetTimeDifferenceofKeyDownDown(data):
    prev=0
    rows=[]
    for index,row in data.iterrows():
        if prev==0 and row[1]=="KeyDown":
            prevRow=row
            prev=1
        elif row[1]=="KeyDown":
            rows.append([ prevRow[0], row[0], int(row[2])- int(prevRow[2]),row[3]])
            prevRow=row
    return rows

In [13]:
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=temp)
keyDownDownTimeDifference


[['I', 'A', 359, '001000'],
 ['A', 'M', 296, '001000'],
 ['M', 'H', 359, '001000'],
 ['H', 'O', 140, '001000'],
 ['O', 'N', 141, '001000'],
 ['N', 'E', 296, '001000'],
 ['E', 'O', 2980, '001000'],
 ['O', 'R', 62, '001000'],
 ['R', 'E', 78, '001000'],
 ['E', 'D', 515, '001000'],
 ['D', 'T', 281, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'B', 250, '001000'],
 ['B', 'E', 187, '001000'],
 ['E', 'W', 343, '001000'],
 ['W', 'I', 140, '001000'],
 ['I', 'T', 63, '001000'],
 ['T', 'H', 1092, '001000'],
 ['H', 'Y', 1357, '001000'],
 ['Y', 'O', 172, '001000'],
 ['O', 'U', 62, '001000'],
 ['U', 'T', 203, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'D', 140, '001000'],
 ['D', 'A', 63, '001000'],
 ['A', 'Y', 343, '001000'],
 ['Y', 'A', 1076, '001000'],
 ['A', 'A', 983, '001000'],
 ['A', 'T', 141, '001000'],
 ['T', 'Y', 374, '001000'],
 ['Y', 'O', 187, '001000'],
 ['O', 'U', 78, '001000'],
 ['U', 'R', 141, '001000'],
 ['R', 'C', 405, '001000'],
 ['C', 'O', 109, '001000'],
 ['O', 'M', 172, '0010

In [14]:
def GetDictionaryOfFeatureVectors(keyDownDownTimeDifference):
    """
    Take List of arrays and return dictionary of 26*26 vector for each user
    """
    dictFV=dict()
    for xi in keyDownDownTimeDifference:
        if xi[3] not in dictFV:
            dictFV[xi[3]]=np.zeros(26*26,dtype=object)
            
        index=(ord(xi[0])-65)*26+np.absolute(ord(xi[1])-65)
        if dictFV[xi[3]][index]==0:
            dictFV[xi[3]][index]=[]
        dictFV[xi[3]][index].append(xi[2])
    return dictFV


In [15]:
FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
usersFV=pd.DataFrame.from_dict(FeatureVectors)
GMMData=[]
for user in usersFV.columns:
    maxLength=0
    for userF in usersFV[user]:
        if userF==0:
            userF=[]
        featureLenght=len(userF)
        if featureLenght > maxLength:
            maxLength=featureLenght
    userDF=pd.DataFrame(index=range(0,676))
    userData=np.zeros((maxLength,676))
    row=0
    for userF in usersFV[user]:
        column=0
        if userF==0:
            userF=[]
        availableValuesCount=len(userF)
        sum=0
        for value in userF:
            userData[column][row]=value
            column=column+1
            sum=sum+value
        if availableValuesCount==0:
            mean=0
        else:
            mean=sum/availableValuesCount
        gussianValues=np.random.normal(mean,3,maxLength-availableValuesCount)
        for value in gussianValues:
            
            userData[column][row]=value
            column=column+1
        
        row=row+1
    print(userData.shape)
    if len(GMMData)==0:
        GMMData=userData
    else:
        GMMData=np.append(GMMData,userData,axis=0)

print(GMMData.shape)

(66, 676)
(48, 676)
(114, 676)


In [16]:
FeatureVectors

{'001000': array([list([983, 609, 1919, 905]), list([171, 266, 156, 172, 219, 187]),
        list([390, 436, 748, 203, 250, 1077, 203, 172, 281]),
        list([249, 266, 1014, 249, 234, 218, 203, 218, 156, 188, 187, 172, 203, 202, 219, 187, 187, 156]),
        list([671, 2278]), list([156, 359, 421]), list([733]),
        list([530, 390]), list([141, 171, 422]), 0,
        list([109, 187, 250, 125, 171, 141]),
        list([265, 156, 156, 171, 670, 93, 156, 156, 156, 125, 109, 171, 203, 140, 141, 140, 141, 172, 156, 125, 156, 172, 202]),
        list([296, 202, 141, 62, 140]),
        list([187, 140, 218, 109, 109, 250, 172, 125, 171, 406, 203, 171, 250, 140, 249, 156, 156, 156, 141, 858, 234, 250, 328, 156, 125, 172, 156, 125, 141]),
        0, list([125, 187, 265, 952, 172, 156]), 0,
        list([156, 110, 109, 109, 187, 218, 124, 125, 140, 125, 483, 686, 141, 63, 500, 94]),
        list([483, 405, 171, 203, 109, 577, 281, 124, 124, 78, 171, 171, 203, 156, 109, 124, 141, 109, 234, 

In [17]:
def GetAverageFeatureVectors(FeatureVector):
    averageFV= np.zeros(26*26,dtype=np.float)
    index=0
    for xi in FeatureVector:
        tempSum=np.array(xi).sum()
        if type(xi)== list:
            averageFV[index]=tempSum/len(xi)
        index=index+1
    return averageFV

In [18]:
X = np.zeros(shape=(len(FeatureVectors),26*26),dtype=object)
index = 0
#X = np.array()
for user in FeatureVectors:
    X[index] = GetAverageFeatureVectors(FeatureVectors[user])
    index = index + 1

In [19]:
X.shape

(2, 676)

In [20]:
Y = np.zeros(len(FeatureVectors),dtype=int)
index = 0
for user in FeatureVectors:
    Y[index] = user
    index = index + 1

In [21]:
Y

array([1000, 1001])

In [22]:
shape = (4, 6)

zeros_array = np.empty(26*26)

In [23]:
zeros_array

array([1.22720190e-311, 1.22726263e-311, 2.87066667e+002, 2.54750000e+002,
       5.58000000e+002, 9.21000000e+002, 4.17333333e+002, 1.56000000e+002,
       9.06333333e+002, 0.00000000e+000, 1.82000000e+002, 2.11894737e+002,
       2.62400000e+002, 1.60428571e+002, 3.43000000e+002, 0.00000000e+000,
       0.00000000e+000, 1.48100000e+002, 2.47000000e+002, 1.69523810e+002,
       3.08250000e+002, 1.51000000e+002, 8.35000000e+002, 0.00000000e+000,
       1.67500000e+002, 9.50000000e+002, 2.02600000e+002, 4.84000000e+002,
       1.18600000e+003, 0.00000000e+000, 2.96375000e+002, 3.27000000e+002,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 2.19000000e+002, 0.00000000e+000, 0.00000000e+000,
       3.74500000e+002, 0.00000000e+000, 0.00000000e+000, 2.88500000e+002,
       1.07000000e+002, 1.39366667e+003, 3.95800000e+002, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       3.90000000e+002, 0

In [150]:
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, Convolution1D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
import keras

In [151]:
total_sample_size = 5
num_of_pairs = 2
dim1 = 26*26
dim2 = 1
x_geuine_pair = np.zeros([total_sample_size, num_of_pairs, 1, dim1])
y_genuine = np.zeros([total_sample_size, 1])


x_imposite_pair = np.zeros([total_sample_size, num_of_pairs, 1, dim1])
y_imposite = np.zeros([total_sample_size, 1])

In [152]:
path1="..\\UB_keystroke_dataset\\S0"
sample1 = ReadListOfTextFilesFromDirectoryWalk(path1)
path2="..\\UB_keystroke_dataset\\S1"
sample2 = ReadListOfTextFilesFromDirectoryWalk(path2)

In [153]:
for count in range(total_sample_size):
    print(sample1[count])
    print(sample2[count])
    items = CreateListOfTuplesFromFile(sample1[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_geuine_pair[count, 0, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    items = CreateListOfTuplesFromFile(sample2[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_geuine_pair[count, 1, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    y_genuine[count] = 1


..\UB_keystroke_dataset\S0\baseline\001000.txt
..\UB_keystroke_dataset\S1\baseline\001100.txt
..\UB_keystroke_dataset\S0\baseline\001001.txt
..\UB_keystroke_dataset\S1\baseline\001101.txt
..\UB_keystroke_dataset\S0\baseline\002000.txt
..\UB_keystroke_dataset\S1\baseline\002100.txt
..\UB_keystroke_dataset\S0\baseline\002001.txt
..\UB_keystroke_dataset\S1\baseline\002101.txt
..\UB_keystroke_dataset\S0\baseline\003000.txt
..\UB_keystroke_dataset\S1\baseline\003100.txt


In [154]:
for count in range(total_sample_size):
    while True:
        ind1 = np.random.randint(total_sample_size)
        ind2 = np.random.randint(total_sample_size)
        if ind1 != ind2:
            break
            
    print(sample1[ind1])
    print(sample2[ind2])

    items = CreateListOfTuplesFromFile(sample1[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_imposite_pair[count, 0, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    items = CreateListOfTuplesFromFile(sample2[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_imposite_pair[count, 1, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    y_imposite[count] = 0


..\UB_keystroke_dataset\S0\baseline\001000.txt
..\UB_keystroke_dataset\S1\baseline\001101.txt
..\UB_keystroke_dataset\S0\baseline\002001.txt
..\UB_keystroke_dataset\S1\baseline\001100.txt
..\UB_keystroke_dataset\S0\baseline\002000.txt
..\UB_keystroke_dataset\S1\baseline\003100.txt
..\UB_keystroke_dataset\S0\baseline\002000.txt
..\UB_keystroke_dataset\S1\baseline\001101.txt
..\UB_keystroke_dataset\S0\baseline\002001.txt
..\UB_keystroke_dataset\S1\baseline\001101.txt


In [155]:
x_imposite_pair.shape

(5, 2, 1, 676)

In [156]:
X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
Y = np.concatenate([y_genuine, y_imposite], axis=0)

In [157]:
Y.shape
X.shape

(10, 2, 1, 676)

In [158]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [173]:
def build_base_network(input_shape):
    
    #seq = Sequential()
    
    
    #nb_filter = [6, 12]
    #kernel_size = 3
    
    
    ##########################
    model_m = Sequential()
  #  model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
    model_m.add(Conv1D(10, 1, activation='relu', input_shape=input_shape))
    model_m.add(Conv1D(10, 1, activation='relu'))
 #   model_m.add(MaxPooling1D(3))
 #   model_m.add(Conv1D(160, 10, activation='relu'))
 #   model_m.add(Conv1D(160, 10, activation='relu'))
    model_m.add(GlobalAveragePooling1D())
    #model_m.add(Flatten())
    model_m.add(Dropout(0.5))
    model_m.add(Dense(50, activation='softmax'))
    print(model_m.summary())
    ##########################

    return model_m

In [174]:
input_dim = x_train.shape[2:]
#print(input_dim)
#img_a = Input(shape=input_dim)
#img_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
#feat_vecs_a = base_network(img_a)
#feat_vecs_b = base_network(img_b)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_31 (Conv1D)           (None, 1, 10)             6770      
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 1, 10)             110       
_________________________________________________________________
global_average_pooling1d_12  (None, 10)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                550       
Total params: 7,430
Trainable params: 7,430
Non-trainable params: 0
_________________________________________________________________
None


In [170]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

base_network.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 5
EPOCHS = 5



In [169]:
history = base_network.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

ValueError: Error when checking input: expected conv1d_29_input to have 3 dimensions, but got array with shape (7, 2, 1, 676)

In [81]:
input_dim

(1, 676, 1)

In [34]:
with open(sample1[count]) as file:
    data = file.read()

In [40]:
items = []
iterator = (sample1[count],) if not isinstance(sample1[count], (tuple, list)) else sample1[count]
for file in iterator:
    filename=file.split('\\')[-1].split('.')[0]
    f=open(file, "r")
    for line in f:
        words= line.rstrip('\n').split(sep=" ")
        items.append((words[0],words[1],words[2],filename))
    f.close()
